In [0]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette-160.tgz

--2019-10-08 10:54:45--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette-160.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.244.110
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.244.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98752094 (94M) [application/x-tar]
Saving to: ‘imagenette-160.tgz’

imagenette-160.tgz  100%[===================>]  94.18M  33.6MB/s    in 2.8s    

2019-10-08 10:54:48 (33.6 MB/s) - ‘imagenette-160.tgz’ saved [98752094/98752094]



In [0]:
!tar -xvzf imagenette-160.tgz

imagenette-160/
imagenette-160/val/
imagenette-160/val/n03028079/
imagenette-160/val/n03028079/ILSVRC2012_val_00034021.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00026451.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00006268.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00004912.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00009705.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00017783.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00033509.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00046232.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00019331.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00005295.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00048853.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00035753.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00004939.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00008116.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00034220.JPEG
imagenette-160/val/n03028079/ILSVRC2012_val_00034534.JPEG
imagen

In [0]:
labels = ["tench", "English springer", "cassette player", "chain saw", "church", "French horn", "garbage truck", "gas pump", "golf ball", "parachute"]

In [1]:
import torch
from torchvision import transforms, datasets

data_transform = transforms.Compose([
        transforms.Resize((64,64)),
        transforms.ToTensor(),
    ])


train_dataset = datasets.ImageFolder(root='imagenette-160/train',
                                           transform=data_transform)

train_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=32, 
                                             shuffle=True,
                                             num_workers=4)


val_dataset = datasets.ImageFolder(root='imagenette-160/val',
                                           transform=data_transform)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=32, 
                                             shuffle=False,
                                             num_workers=4)



train_dataset[0][0].shape
train_dataset[1600][1]
print(len(train_dataset))
train_dataset.classes

12894


['n01440764',
 'n02102040',
 'n02979186',
 'n03000684',
 'n03028079',
 'n03394916',
 'n03417042',
 'n03425413',
 'n03445777',
 'n03888257']

In [0]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.layers = nn.Sequential(
                      nn.Conv2d(3, 6, kernel_size=3),
                      nn.MaxPool2d(kernel_size=2, stride=2),
                      nn.Conv2d(6, 16, kernel_size=5),
                      nn.MaxPool2d(kernel_size=2, stride=2),            
                      )

        self.fc1 = torch.nn.Linear(16*13*13, 128)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.layers(x)
        x = x.view(-1, 16*13*13)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return x
      
model = SimpleCNN().cuda()


In [0]:
## INSTRUCTIONS:

#Utiliser les datasets et dalaloader pour utiliser les données (ici, on utilise ImageFolder)
#Utiliser la fonction transform pour pour resize les images 64,64
#On envoit les données par paquet dans le réseau (batch_size), a vous de trouver le bon nombre a donner en argument a votre dataloader

#Utiliser le GPU : appeler cuda() sur votre modèle (qui implémente torch.nn.module) et sur les données qui entrent dans votre réseau

#Bonne pratique pour les CNN : faire une succession de convolution (nn.conv2d), d'activation relu (F.relu), puis de pooling (nn.MaxPool2d)
#Attention, les features maps du CNN sont en 3D, il faut repasser a 1D à un moment pour la classif , utiliser torch.view (ex : batch_size,3x3x64 => bathc_size,576 puis bs,576 -> bs,10)
#La sortie doit etre de taille 10, il y a 10 classes

#Utiliser la loss adaptée pour classif
#Utiliser la descente de gradient stochastique pour optimiser le reseau (optim.SGD) 
#Avant d'envoyer un paquet dans le réseau, utiliser optimize.zero_grad. Après passage du batch, utiliser optim.step()
#A vous de trouver le meilleur learning rate

#Apres chaque epoch, faire une evaluation
#Prediction : prendre à la sortie la classe qui a la meilleure confiance (torch.max) et comparer avec le label ground-truth. Pour cela, il faut itérer sur le dataloader de val.

#Restrictions :
# 1)
#Avoir moins de 500.000 parametres
# print("Nombre total de params",sum(p.numel() for p in model.parameters())) ou la variable model est votre modèle.

# 2)
#Avoir moins de 250 secondes d'execution :
# import time
# start = time.time()
# A la fin : print("Temps d'execution : ", time.time()-start, 'secondes.')  


#Notation : 1/1 si réseau finit ses 10 epochs en moins de 250 secondes avec une epoch avec précision > 55 %
#           1 point bonus si vous battez ma solution en temps (145 secondes) ET en précision (65%)
#Ma solution :

In [6]:
import time
start = time.time()

model = SimpleCNN().cuda()
#Loss function
loss_func = torch.nn.CrossEntropyLoss()    
#Optimizer
optimizer = torch.optim.SGD(model.parameters(),  lr=0.003, momentum=0.9)
    
print("Nombre total de params",sum(p.numel() for p in model.parameters()))

num_epoch = 10
loss_epoch = 0
for epoch in range(num_epoch):
    for i, (x, y) in enumerate(train_loader):   # gives batch data
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      out = model(x)
      loss = loss_func(out, y)
      loss.backward()
      optimizer.step()
      loss_epoch += loss.data

    print("Loss for epoch",epoch,":",loss_epoch.data)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
      for i, (x, y) in enumerate(val_loader):
          x = x.cuda()
          y = y.cuda()
          
          outputs = model(x)
          _, predicted = torch.max(outputs.data, 1)
          total += y.size(0)
          correct += (predicted == y).sum().item()

      print('Precision du réseau sur les données val: %d %%' % (
          100 * correct / total))
      
    model.train()
    loss_epoch = 0
           

print("Temps d'execution : ", time.time()-start, 'secondes.')      
      

Nombre total de params 350114
Loss for epoch 0 : tensor(840.3370, device='cuda:0')
Precision du réseau sur les données val: 40 %
Loss for epoch 1 : tensor(698.0801, device='cuda:0')
Precision du réseau sur les données val: 48 %
Loss for epoch 2 : tensor(619.6754, device='cuda:0')
Precision du réseau sur les données val: 54 %
Loss for epoch 3 : tensor(563.1716, device='cuda:0')
Precision du réseau sur les données val: 57 %
Loss for epoch 4 : tensor(505.4431, device='cuda:0')
Precision du réseau sur les données val: 61 %
Loss for epoch 5 : tensor(453.9288, device='cuda:0')
Precision du réseau sur les données val: 65 %
Loss for epoch 6 : tensor(397.5204, device='cuda:0')
Precision du réseau sur les données val: 62 %
Loss for epoch 7 : tensor(338.5242, device='cuda:0')
Precision du réseau sur les données val: 61 %
Loss for epoch 8 : tensor(279.8197, device='cuda:0')
Precision du réseau sur les données val: 61 %
Loss for epoch 9 : tensor(221.0976, device='cuda:0')
Precision du réseau sur le